In [1]:
import uncertainties 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.optimize import curve_fit
from statistics import stdev
from scipy import odr
from scipy.odr import RealData
from uncertainties import ufloat
from uncertainties.umath import *
from scipy import optimize
print(Path.cwd().absolute())

C:\Users\lukso\Dropbox\Praktika I


In [5]:
# Reynolds 
Typ_oleje = ["ricínový","Parafínový","Parafínový"]
viscoz = np.array([986,101.8,101.8])*1e-3
rho = np.array([950,850,850])
diam = np.array([2.8,1.13,2.8])*1e-3
Re = np.pi*rho*diam/viscoz
dictionary = {"Olej":Typ_oleje,
             "eta":viscoz,
             "rho":rho,
             "d":diam,
              "v":vel,
              "Re": Re
             }

array([ 8.47529053, 29.64145131, 73.44784396])

In [109]:
# hustoty
mp = [18.1317,18.1317]
m_1 = [21.4355,21.0114]
m_2 = [28.3370,2.8337]
m_3 = [30.4170,30.0925]
rho = [2691,2553]

rho_eval = {"mp":mp,
         "m_1":m_1,
         "m_2":m_2,
         "m_3":m_3,
         "res_rho":rho
         } 
rho_frame = pd.DataFrame(rho_eval)

with open("Tex_tables/XIXrho.tex","w") as f:
    f.write(rho_frame.to_latex(index_names=False, index=False,na_rep="-"))

C:\Users\lukso\AppData\Local\Temp\ipykernel_4884\3338604278.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(rho_frame.to_latex(index_names=False, index=False,na_rep="-"))


In [6]:
2.8*986/986

2.8

In [124]:
#tabulka na hodnoty
d_v = np.array([1.09,0.87,1.23,1.27,1.17,1.12,1.11,1.09,1.17,1.22])
d_m = np.array([2.89,2.84,2.84,2.75,2.75,2.77,2.8,2.78,2.76,2.86,2.72])
d_vmean = np.mean(d_v)
d_mmean = np.mean(d_m,)

print(d_vmean,d_mmean)
# je třeba odebrat řádek prázdnej v latexu
table = {"Typ kuličky":["Velká","Malá"]
         
}
for n in range(len(d_v)):
    table[str(n+1)] = np.array([d_v[n],d_m[n]])
table["mu"] = np.array([d_vmean,d_mmean])
table["sigma"] = np.array([np.std(d_v),np.std(d_m)])
tabulka =  pd.DataFrame(table)

with open("Tex_tables/XIXd.tex","w") as f:
    f.write(tabulka.to_latex(index_names=False, index=False,na_rep="-"))

1.134 2.7963636363636364


,Typ kuličky,1,2,3,4,5,6,7,8,9,10,mu,sigma
0,Velká,1.09,0.87,1.23,1.27,1.17,1.12,1.11,1.09,1.17,1.22,1.134000,0.105849
1,Malá,2.89,2.84,2.84,2.75,2.75,2.77,2.80,2.78,2.76,2.86,2.796364,0.051394


In [132]:
# časy
casy = ["t1","t2"]
t1 = np.array([4.19,5.41,4.57,5.41,5.34,6.85])
t2 = np.array([21.16,23.94,21.13,22.64,24.91,26.52])
t_mean = np.array([np.mean(t1),np.mean(t2)])
print(t_mean)
tstdev = np.array([np.std(t1),np.std(t2)])
print(tstdev)
# results = {"t1":t1,"t2":t2,"t1mean":mean[0]}

table2 = {"Čas měření":casy
         
}

for n in range(len(t1)):
    table[str(n+1)] = np.array([t1[n],t2[n]])

table2["mu"] = t_mean
table2["sigma"] = tstdev

with open("Tex_tables/XIXt.tex","w") as f:
    f.write(tabulka.to_latex(index_names=False, index=False,na_rep="-"))
# reynolds parametry
Parametry = ["d","d2","m1","m2","rhos1","rhos2"]
hodnoty = [1.13,2.80,18.1317,18.1317,2691,2553]
chyba = [1e-5,5e-5,1e-3,1e-4,1e-2,3,3]
print(t_mean)
ball_param = {
              "s":["mu1","mu2","sigma1","sigma2"],
              "dk [mm]":np.array([d_mmean,d_vmean,np.std(d_m),np.std(d_v)]),
              "lv [cm]":np.array([20.6,20.8,0.3,0.3]),
              "Dv [mm]":np.array([61.6,61.25,0.3,0.2]),
              "g [ms-2]":np.array([9.81,9.81,0.1,0.1]),
              "rhok [kg m-3]":np.array([rho_eval["res_rho"][1],rho_eval["res_rho"][0],3,3]),
              "rhol [kg m-3]":np.array([950,850,1,1]),
              "t [s]":np.array([t_mean[1],t_mean[0],tstdev[1],tstdev[0]])
            
}
g = ufloat(9.81,1)
g.std_dev
res_frame = pd.DataFrame(ball_param)
Re = np.zeros(4)
eta = np.zeros(4)

n = 2 # počet měření
for x in range(n):
    d = ufloat(res_frame["dk [mm]"][x],res_frame["dk [mm]"][x+n])*1e-3
    rhok = ufloat(res_frame["rhok [kg m-3]"][x],res_frame["rhok [kg m-3]"][x+n])
    rhol = ufloat(res_frame["rhol [kg m-3]"][x],res_frame["rhol [kg m-3]"][x+n])
    g = ufloat(res_frame["g [ms-2]"][x],res_frame["g [ms-2]"][x+n])
    l = ufloat(res_frame["lv [cm]"][x],res_frame["lv [cm]"][x+n])*1e-2
    t = ufloat(res_frame["t [s]"][x],res_frame["t [s]"][x+n])
    D = ufloat(res_frame["Dv [mm]"][x],res_frame["Dv [mm]"][x+n])*1e-3 
    v = l/t
    r = d/2
    R = D/2
    etas = ((2*r**2) *(rhok-rhol)*g)/(9*v*(1-2.4*r/R))
    Ren = 2*np.pi*rhol*v/etas
    print(eta)
    Re[x] = Ren.nominal_value
    eta[x] = etas.nominal_value
    eta[x+n] = etas.std_dev
    Re[x+n] = Ren.std_dev
    print("chyba v procentech")
    print(etas.std_dev/etas.nominal_value *1e2)
print(Re)
res_frame["Re"] = Re 
res_frame["eta"] = eta
# Viscosity
Typ_oleje = ["ricínový","Parafínový","Parafínový"]
viscoz = np.array([986,101.8,101.8])*1e-3
rho = np.array([950,850,850])
diam = np.array([2.8,1.13,2.8])*1e-3
Re = np.pi*rho*diam/viscoz
dictionary = {"Olej":Typ_oleje,
             "eta":viscoz,
             "rho":rho,
             "d":diam,
              # "v":vel,
              "Re": Re
             }
with open("Tex_tables/XIXres.tex","w") as f:
    f.write(res_frame.to_latex(index_names=False, index=False,na_rep="-"))

[ 5.295      23.38333333]
[0.83597747 1.96012471]
[ 5.295      23.38333333]
[0. 0. 0. 0.]
chyba v procentech
9.417199720291244
[0.87027011 0.         0.08195507 0.        ]
chyba v procentech
24.84570583465576
[  60.42410204 6103.48236488   10.5677059  2260.23781517]


C:\Users\lukso\AppData\Local\Temp\ipykernel_4884\3002960025.py:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(tabulka.to_latex(index_names=False, index=False,na_rep="-"))
C:\Users\lukso\AppData\Local\Temp\ipykernel_4884\3002960025.py:83: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(res_frame.to_latex(index_names=False, index=False,na_rep="-"))


In [133]:
res_frame

,s,dk [mm],lv [cm],Dv [mm],g [ms-2],rhok [kg m-3],rhol [kg m-3],t [s],Re,eta
0,mu1,2.796364,20.6,61.60,9.81,2553,950,23.383333,60.424102,0.870270
1,mu2,1.134000,20.8,61.25,9.81,2691,850,5.295000,6103.482365,0.034373
2,sigma1,0.051394,0.3,0.30,0.10,3,1,1.960125,10.567706,0.081955
3,sigma2,0.105849,0.3,0.20,0.10,3,1,0.835977,2260.237815,0.008540
